# 농산물 가격 예측을 위한 AI 모델 개발 
- '2024 농산물 가격 예측 AI 경진대회'는 데이터와 AI 기술을 활용하여 농산물 가격 예측 능력을 향상시키는 것을 목표로 합니다.<br>  이 대회는 농업 분야의 복잡한 시계열 데이터를 효율적으로 분석하고 예측할 수 있는 AI 알고리즘 개발에 초점을 맞추고 있습니다. <br> <br>
- 이 대회의 궁극적 목적은 참가자들의 시계열 데이터 분석 및 예측 역량을 강화하고, <br> AI 기술이 실제 농산물 가격 예측과 관련 정책 결정에 어떻게 기여할 수 있는지 탐구하는 것입니다. 

# Import Library

In [1]:
import pandas as pd
import numpy as np
import torch
import torch.nn as nn
from tqdm.notebook import tqdm
from torch.utils.data import Dataset, DataLoader
from sklearn.model_selection import train_test_split
from types import SimpleNamespace
from sklearn.preprocessing import MinMaxScaler
import os

# Hyperparameter Setting

In [2]:
config = {
    "learning_rate": 2e-5,
    "epoch": 30,
    "batch_size": 64,
    "hidden_size": 64,
    "num_layers": 2,
    "output_size": 3
}

CFG = SimpleNamespace(**config)

품목_리스트 = ['건고추', '사과', '감자', '배', '깐마늘(국산)', '무', '상추', '배추', '양파', '대파']

# Define Function for Feature Engineering
- 타겟의 필터 조건을 제외한 메타데이터의 필터 조건은 참가자들 각자의 기준에 맞춰 자유롭게 사용가능 
- 밑의 필터 조건은 임의로 제공하는 예시

In [3]:

def process_data(raw_file, 산지공판장_file, 전국도매_file, 품목명, scaler=None):
    raw_data = pd.read_csv(raw_file)
    산지공판장 = pd.read_csv(산지공판장_file)
    전국도매 = pd.read_csv(전국도매_file)

    # 타겟 및 메타데이터 필터 조건 정의
    conditions = {
    '감자': {
        'target': lambda df: (df['품종명'] == '감자 수미') & (df['거래단위'] == '20키로상자') & (df['등급'] == '상'),
        '공판장': {'공판장명': ['*전국농협공판장'], '품목명': ['감자'], '품종명': ['수미'], '등급명': ['상']},
        '도매': {'시장명': ['*전국도매시장'], '품목명': ['감자'], '품종명': ['수미']}
    },
    '건고추': {
        'target': lambda df: (df['품종명'] == '화건') & (df['거래단위'] == '30 kg') & (df['등급'] == '상품'),
        '공판장': None, 
        '도매': None  
    },
    '깐마늘(국산)': {
        'target': lambda df: (df['거래단위'] == '20 kg') & (df['등급'] == '상품'),
        '공판장': {'공판장명': ['*전국농협공판장'], '품목명': ['마늘'], '품종명': ['깐마늘'], '등급명': ['상']},
        '도매': {'시장명': ['*전국도매시장'], '품목명': ['마늘'], '품종명': ['깐마늘']}
    },
    '대파': {
        'target': lambda df: (df['품종명'] == '대파(일반)') & (df['거래단위'] == '1키로단') & (df['등급'] == '상'),
        '공판장': {'공판장명': ['*전국농협공판장'], '품목명': ['대파'], '품종명': ['대파(일반)'], '등급명': ['상']},
        '도매': {'시장명': ['*전국도매시장'], '품목명': ['대파'], '품종명': ['대파(일반)']}
    },
    '무': {
        'target': lambda df: (df['거래단위'] == '20키로상자') & (df['등급'] == '상'),
        '공판장': {'공판장명': ['*전국농협공판장'], '품목명': ['무'], '품종명': ['기타무'], '등급명': ['상']},
        '도매': {'시장명': ['*전국도매시장'], '품목명': ['무'], '품종명': ['무']}
    },
    '배추': {
        'target': lambda df: (df['거래단위'] == '10키로망대') & (df['등급'] == '상'),
        '공판장': {'공판장명': ['*전국농협공판장'], '품목명': ['배추'], '품종명': ['쌈배추'], '등급명': ['상']},
        '도매': {'시장명': ['*전국도매시장'], '품목명': ['배추'], '품종명': ['배추']}
    },
    '사과': {
        'target': lambda df: (df['품종명'].isin(['홍로', '후지'])) & (df['거래단위'] == '10 개') & (df['등급'] == '상품'),
        '공판장': {'공판장명': ['*전국농협공판장'], '품목명': ['사과'], '품종명': ['후지'], '등급명': ['상']},
        '도매': {'시장명': ['*전국도매시장'], '품목명': ['사과'], '품종명': ['후지']}
    },
    '상추': {
        'target': lambda df: (df['품종명'] == '청') & (df['거래단위'] == '100 g') & (df['등급'] == '상품'),
        '공판장': {'공판장명': ['*전국농협공판장'], '품목명': ['상추'], '품종명': ['청상추'], '등급명': ['상']},
        '도매': {'시장명': ['*전국도매시장'], '품목명': ['상추'], '품종명': ['청상추']}
    },
    '양파': {
        'target': lambda df: (df['품종명'] == '양파') & (df['거래단위'] == '1키로') & (df['등급'] == '상'),
        '공판장': {'공판장명': ['*전국농협공판장'], '품목명': ['양파'], '품종명': ['기타양파'], '등급명': ['상']},
        '도매': {'시장명': ['*전국도매시장'], '품목명': ['양파'], '품종명': ['양파(일반)']}
    },
    '배': {
        'target': lambda df: (df['품종명'] == '신고') & (df['거래단위'] == '10 개') & (df['등급'] == '상품'),
        '공판장': {'공판장명': ['*전국농협공판장'], '품목명': ['배'], '품종명': ['신고'], '등급명': ['상']},
        '도매': {'시장명': ['*전국도매시장'], '품목명': ['배'], '품종명': ['신고']}
    }
    }

    # 타겟 데이터 필터링
    raw_품목 = raw_data[raw_data['품목명'] == 품목명]
    target_mask = conditions[품목명]['target'](raw_품목)
    filtered_data = raw_품목[target_mask]

    # 다른 품종에 대한 파생변수 생성
    other_data = raw_품목[~target_mask]
    unique_combinations = other_data[['품종명', '거래단위', '등급']].drop_duplicates()
    for _, row in unique_combinations.iterrows():
        품종명, 거래단위, 등급 = row['품종명'], row['거래단위'], row['등급']
        mask = (other_data['품종명'] == 품종명) & (other_data['거래단위'] == 거래단위) & (other_data['등급'] == 등급)
        temp_df = other_data[mask]
        for col in ['평년 평균가격(원)', '평균가격(원)']:
            new_col_name = f'{품종명}_{거래단위}_{등급}_{col}'
            filtered_data = filtered_data.merge(temp_df[['시점', col]], on='시점', how='left', suffixes=('', f'_{new_col_name}'))
            filtered_data.rename(columns={f'{col}_{new_col_name}': new_col_name}, inplace=True)


    # 공판장 데이터 처리
    if conditions[품목명]['공판장']:
        filtered_공판장 = 산지공판장
        for key, value in conditions[품목명]['공판장'].items():
            filtered_공판장 = filtered_공판장[filtered_공판장[key].isin(value)]
        
        filtered_공판장 = filtered_공판장.add_prefix('공판장_').rename(columns={'공판장_시점': '시점'})
        filtered_data = filtered_data.merge(filtered_공판장, on='시점', how='left')

    # 도매 데이터 처리
    if conditions[품목명]['도매']:
        filtered_도매 = 전국도매
        for key, value in conditions[품목명]['도매'].items():
            filtered_도매 = filtered_도매[filtered_도매[key].isin(value)]
        
        filtered_도매 = filtered_도매.add_prefix('도매_').rename(columns={'도매_시점': '시점'})
        filtered_data = filtered_data.merge(filtered_도매, on='시점', how='left')

    # 수치형 컬럼 처리
    numeric_columns = filtered_data.select_dtypes(include=[np.number]).columns
    filtered_data = filtered_data[['시점'] + list(numeric_columns)]
    filtered_data[numeric_columns] = filtered_data[numeric_columns].fillna(0)

    # 정규화 적용
    if scaler is None:
        scaler = MinMaxScaler()
        filtered_data[numeric_columns] = scaler.fit_transform(filtered_data[numeric_columns])
    else:
        filtered_data[numeric_columns] = scaler.transform(filtered_data[numeric_columns])

    return filtered_data, scaler


# Define Custom Dataset Class

In [5]:
class AgriculturePriceDataset(Dataset):
    def __init__(self, dataframe, window_size=9, prediction_length=3, is_test=False):
        self.data = dataframe
        self.window_size = window_size
        self.prediction_length = prediction_length
        self.is_test = is_test
        
        self.price_column = '평균가격(원)'
        self.numeric_columns = self.data.select_dtypes(include=[np.number]).columns.tolist()
        
        self.sequences = []
        if not self.is_test:
            for i in range(len(self.data) - self.window_size - self.prediction_length + 1):
                x = self.data[self.numeric_columns].iloc[i:i+self.window_size].values
                y = self.data[self.price_column].iloc[i+self.window_size:i+self.window_size+self.prediction_length].values
                self.sequences.append((x, y))
        else:
            self.sequences = [self.data[self.numeric_columns].values]
    
    def __len__(self):
        return len(self.sequences)
    
    def __getitem__(self, idx):
        if not self.is_test:
            x, y = self.sequences[idx]
            return torch.FloatTensor(x), torch.FloatTensor(y)
        else:
            return torch.FloatTensor(self.sequences[idx])

In [7]:
품목별_predictions = {}
품목별_scalers = {}

pbar_outer = tqdm(품목_리스트, desc="품목 처리 중", position=0)
for 품목명 in pbar_outer:
    pbar_outer.set_description(f"품목별 전처리 및 모델 학습 -> {품목명}")
    train_data, scaler = process_data("./train/train.csv", 
                              "./train/meta/TRAIN_산지공판장_2018-2021.csv", 
                              "./train/meta/TRAIN_전국도매_2018-2021.csv", 
                              품목명)
    품목별_scalers[품목명] = scaler
    dataset = AgriculturePriceDataset(train_data)

품목 처리 중:   0%|          | 0/10 [00:00<?, ?it/s]

In [17]:
train_data

,시점,평년 평균가격(원),평균가격(원),실파_20키로_중_평년 평균가격(원),실파_20키로_중_평균가격(원),깐쪽파_10키로상자_상_평년 평균가격(원),깐쪽파_10키로상자_상_평균가격(원),깐쪽파_10키로상자_중_평년 평균가격(원),깐쪽파_10키로상자_중_평균가격(원),쪽파_10키로상자_특_평년 평균가격(원),...,도매_저가(20%) 평균가,도매_중간가(원/kg),도매_최저가(원/kg),도매_최고가(원/kg),도매_경매 건수,도매_전순 평균가격(원) PreVious SOON,도매_전달 평균가격(원) PreVious MMonth,도매_전년 평균가격(원) PreVious YeaR,도매_평년 평균가격(원) Common Year SOON,도매_연도
0,201801상순,0.453252,0.206004,0.0,0.0,0.524720,0.196302,0.444128,0.197356,0.732697,...,0.151546,0.162696,0.0,0.005096,0.251326,0.447152,1.000000,0.262850,0.786162,0.0
1,201801중순,0.303121,0.193396,0.0,0.0,0.434956,0.299988,0.404316,0.304934,0.506358,...,0.173660,0.174949,0.0,0.005215,0.324770,0.173156,0.237672,0.231132,0.839018,0.0
2,201801하순,0.377425,0.237118,0.0,0.0,0.648365,0.182448,0.613675,0.175447,0.637069,...,0.158179,0.183118,0.0,0.006976,0.342642,0.170789,0.230677,0.238988,0.576211,0.0
3,201802상순,0.585045,0.382363,0.0,0.0,0.610305,0.225140,0.616934,0.224204,0.665802,...,0.289609,0.326072,0.0,0.007602,0.472214,0.185754,0.089328,0.241376,0.961160,0.0
4,201802중순,0.725136,0.175224,0.0,0.0,0.623957,0.217061,0.632647,0.197530,0.665476,...,0.111608,0.216940,0.0,0.002426,0.146328,0.334845,0.088107,0.213656,0.887274,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
139,202111중순,0.692519,0.220395,0.0,0.0,0.357869,0.372143,0.352983,0.334557,0.381547,...,0.165616,0.149847,0.0,0.014212,0.767383,0.161949,0.073495,0.140203,0.070874,1.0
140,202111하순,0.570966,0.159028,0.0,0.0,0.274652,0.239767,0.230745,0.191847,0.276568,...,0.105059,0.091219,0.0,0.002590,0.644513,0.188862,0.080109,0.107443,0.059464,1.0
141,202112상순,0.535608,0.192167,0.0,0.0,0.275106,0.212641,0.238730,0.167095,0.417757,...,0.136960,0.125200,0.0,0.000000,0.635297,0.117882,0.083546,0.078192,0.092443,1.0
142,202112중순,0.497044,0.108275,0.0,0.0,0.432078,0.233311,0.428305,0.166083,0.743859,...,0.087985,0.078536,0.0,0.001656,0.433119,0.153135,0.097431,0.084946,0.062201,1.0


In [14]:
pbar_inner = tqdm(range(25), desc="테스트 파일 추론 중", position=1, leave=False)
for i in pbar_inner:
    test_file = f"./test/TEST_{i:02d}.csv"
    산지공판장_file = f"./test/meta/TEST_산지공판장_{i:02d}.csv"
    전국도매_file = f"./test/meta/TEST_전국도매_{i:02d}.csv"

    test_data, _ = process_data(test_file, 산지공판장_file, 전국도매_file, 품목명, scaler=품목별_scalers[품목명])
    test_dataset = AgriculturePriceDataset(test_data, is_test=True)
test_dataset

테스트 파일 추론 중:   0%|          | 0/25 [00:00<?, ?it/s]

In [16]:
test_data

,시점,평년 평균가격(원),평균가격(원),실파_20키로_중_평년 평균가격(원),실파_20키로_중_평균가격(원),깐쪽파_10키로상자_상_평년 평균가격(원),깐쪽파_10키로상자_상_평균가격(원),깐쪽파_10키로상자_중_평년 평균가격(원),깐쪽파_10키로상자_중_평균가격(원),쪽파_10키로상자_특_평년 평균가격(원),...,도매_저가(20%) 평균가,도매_중간가(원/kg),도매_최저가(원/kg),도매_최고가(원/kg),도매_경매 건수,도매_전순 평균가격(원) PreVious SOON,도매_전달 평균가격(원) PreVious MMonth,도매_전년 평균가격(원) PreVious YeaR,도매_평년 평균가격(원) Common Year SOON,도매_연도
0,T-8순,0.424166,0.149015,0.000000,0.000000,0.221410,0.355942,0.224259,0.375217,0.436762,...,0.170128,0.147360,0.0,0.008542,0.104999,0.102323,0.074121,0.363346,0.076999,1.333333
1,T-7순,0.543494,0.219027,0.000000,0.000000,0.081352,0.248016,0.079776,0.263547,0.211048,...,0.233076,0.208646,0.0,0.010277,0.256632,0.147608,0.056328,0.335728,0.095148,1.333333
2,T-6순,0.405021,0.167587,0.000000,0.000000,0.078419,0.087512,0.078622,0.089443,0.143002,...,0.154588,0.152485,0.0,0.006349,0.182351,0.217372,0.052787,0.261096,0.062830,1.333333
3,T-5순,0.327338,0.127621,0.000000,0.000000,0.036809,0.068250,0.037493,0.066321,0.100440,...,0.111400,0.101430,0.0,0.010277,0.373639,0.158049,0.076149,0.266429,0.054651,1.333333
4,T-4순,0.204617,0.089957,0.000000,0.000000,0.020027,0.079542,0.023121,0.075046,0.081288,...,0.076183,0.063989,0.0,0.003216,0.219492,0.104373,0.112138,0.257008,0.039196,1.333333
5,T-3순,0.048652,0.092291,0.000000,0.000000,0.089633,0.121257,0.079174,0.112662,0.133678,...,0.079008,0.062442,0.0,0.008103,0.310249,0.069909,0.081535,0.257704,0.010608,1.333333
6,T-2순,0.084330,0.098090,0.506923,0.337419,0.189668,0.167846,0.177044,0.157535,0.234493,...,0.107491,0.091687,0.0,0.006975,0.310249,0.077904,0.053844,0.192320,0.000000,1.333333
7,T-1순,0.261489,0.148382,0.445156,0.357940,0.233956,0.219715,0.246893,0.227106,0.376993,...,0.138285,0.140222,0.0,0.008949,0.293493,0.097025,0.036065,0.180984,0.023298,1.333333
8,T,0.297255,0.130864,0.354206,0.324466,0.378107,0.311023,0.357789,0.329327,0.539767,...,0.130571,0.130074,0.0,0.008949,0.386484,0.140363,0.040189,0.104064,0.022232,1.333333


# Define Model Architecture and Training Functions

In [ ]:

class PricePredictionLSTM(nn.Module):
    def __init__(self, input_size, hidden_size, num_layers, output_size):
        super(PricePredictionLSTM, self).__init__()
        self.hidden_size = hidden_size
        self.num_layers = num_layers
        self.lstm = nn.LSTM(input_size, hidden_size, num_layers, batch_first=True)
        self.fc = nn.Linear(hidden_size, output_size)
    
    def forward(self, x):
        h0 = torch.zeros(self.num_layers, x.size(0), self.hidden_size).to(x.device)
        c0 = torch.zeros(self.num_layers, x.size(0), self.hidden_size).to(x.device)
        out, _ = self.lstm(x, (h0, c0))
        out = self.fc(out[:, -1, :])
        return out

def train_model(model, train_loader, criterion, optimizer, num_epochs):
    model.train()
    total_loss = 0
    for batch_x, batch_y in train_loader:
        optimizer.zero_grad()
        outputs = model(batch_x)
        loss = criterion(outputs, batch_y)
        loss.backward()
        optimizer.step()
        total_loss += loss.item()
    return total_loss / len(train_loader)  

def evaluate_model(model, test_loader, criterion):
    model.eval()
    total_loss = 0
    with torch.no_grad():
        for batch_x, batch_y in test_loader:
            outputs = model(batch_x)
            loss = criterion(outputs, batch_y)
            total_loss += loss.item()
    return total_loss / len(test_loader)

# Train Models and Generate Predictions

In [ ]:
품목별_predictions = {}
품목별_scalers = {}

pbar_outer = tqdm(품목_리스트, desc="품목 처리 중", position=0)
for 품목명 in pbar_outer:
    pbar_outer.set_description(f"품목별 전처리 및 모델 학습 -> {품목명}")
    train_data, scaler = process_data("./train/train.csv", 
                              "./train/meta/TRAIN_산지공판장_2018-2021.csv", 
                              "./train/meta/TRAIN_전국도매_2018-2021.csv", 
                              품목명)
    품목별_scalers[품목명] = scaler
    dataset = AgriculturePriceDataset(train_data)

    # 데이터를 train과 validation으로 분할
    train_data, val_data = train_test_split(dataset, test_size=0.2, random_state=42)
    
    train_loader = DataLoader(train_data, CFG.batch_size, shuffle=True)
    val_loader = DataLoader(val_data, CFG.batch_size, shuffle=False)

    input_size = len(dataset.numeric_columns)
    
    model = PricePredictionLSTM(input_size, CFG.hidden_size, CFG.num_layers, CFG.output_size)
    criterion = nn.L1Loss()
    optimizer = torch.optim.Adam(model.parameters(), CFG.learning_rate)
    
    best_val_loss = float('inf')
    os.makedirs('models', exist_ok=True)

    for epoch in range(CFG.epoch):
        train_loss = train_model(model, train_loader, criterion, optimizer, CFG.epoch)
        val_loss = evaluate_model(model, val_loader, criterion)
        
        if val_loss < best_val_loss:
            best_val_loss = val_loss
            torch.save(model.state_dict(), f'models/best_model_{품목명}.pth')
        
        print(f'Epoch {epoch+1}/{CFG.epoch}, Train Loss: {train_loss:.4f}, Val Loss: {val_loss:.4f}')
    
    print(f'Best Validation Loss for {품목명}: {best_val_loss:.4f}')
    
    품목_predictions = []

    ### 추론 
    pbar_inner = tqdm(range(25), desc="테스트 파일 추론 중", position=1, leave=False)
    for i in pbar_inner:
        test_file = f"./test/TEST_{i:02d}.csv"
        산지공판장_file = f"./test/meta/TEST_산지공판장_{i:02d}.csv"
        전국도매_file = f"./test/meta/TEST_전국도매_{i:02d}.csv"
        
        test_data, _ = process_data(test_file, 산지공판장_file, 전국도매_file, 품목명, scaler=품목별_scalers[품목명])
        test_dataset = AgriculturePriceDataset(test_data, is_test=True)
        test_loader = DataLoader(test_dataset, batch_size=1, shuffle=False)

        model.eval()
        predictions = []
        with torch.no_grad():
            for batch in test_loader:
                output = model(batch)
                predictions.append(output.numpy())
        
        predictions_array = np.concatenate(predictions)

        # 예측값을 원래 스케일로 복원
        price_column_index = test_data.columns.get_loc(test_dataset.price_column)
        predictions_reshaped = predictions_array.reshape(-1, 1)
        
        # 가격 열에 대해서만 inverse_transform 적용
        price_scaler = MinMaxScaler()
        price_scaler.min_ = 품목별_scalers[품목명].min_[price_column_index]
        price_scaler.scale_ = 품목별_scalers[품목명].scale_[price_column_index]
        predictions_original_scale = price_scaler.inverse_transform(predictions_reshaped)
        #print(predictions_original_scale)
        
        if np.isnan(predictions_original_scale).any():
            pbar_inner.set_postfix({"상태": "NaN"})
        else:
            pbar_inner.set_postfix({"상태": "정상"})
            품목_predictions.extend(predictions_original_scale.flatten())

            
    품목별_predictions[품목명] = 품목_predictions
    pbar_outer.update(1)


# Prepare Submission File

In [33]:
targets = [i for i in train_data.columns if i[:10] == '대파(일반)_1키로']
train_data.loc[:,targets]

,대파(일반)_1키로단_하_평년 평균가격(원),대파(일반)_1키로단_하_평균가격(원),대파(일반)_1키로단_특_평년 평균가격(원),대파(일반)_1키로단_특_평균가격(원),대파(일반)_1키로단_중_평년 평균가격(원),대파(일반)_1키로단_중_평균가격(원)
0,0.628372,0.210856,0.454556,0.223517,0.482437,0.186863
1,0.461933,0.193582,0.289120,0.197761,0.330670,0.169164
2,0.537270,0.204984,0.311956,0.273281,0.384110,0.184032
3,0.734858,0.344545,0.521612,0.524398,0.584103,0.337778
4,0.838963,0.154216,0.685506,0.241997,0.719321,0.144494
...,...,...,...,...,...,...
139,0.667760,0.215352,0.811785,0.257775,0.644319,0.206775
140,0.476548,0.115087,0.799264,0.223151,0.478871,0.123836
141,0.493795,0.164744,0.818470,0.224530,0.471081,0.169141
142,0.555696,0.098464,0.681295,0.102047,0.457239,0.101239


In [36]:
test_data.loc[:,targets]

,대파(일반)_1키로단_하_평년 평균가격(원),대파(일반)_1키로단_하_평균가격(원),대파(일반)_1키로단_특_평년 평균가격(원),대파(일반)_1키로단_특_평균가격(원),대파(일반)_1키로단_중_평년 평균가격(원),대파(일반)_1키로단_중_평균가격(원)
0,0.457043,0.179320,0.513709,0.153450,0.360979,0.153374
1,0.675587,0.244314,0.667848,0.220335,0.528224,0.224491
2,0.444389,0.160995,0.450549,0.175976,0.347581,0.163293
3,0.346974,0.111270,0.321086,0.179809,0.265759,0.105214
4,0.215130,0.076411,0.209116,0.119337,0.152720,0.066226
5,0.043416,0.101306,0.000000,0.110772,0.028319,0.087623
6,0.000000,0.107093,0.038009,0.093533,0.019476,0.102466
7,0.091755,0.129217,0.209513,0.134317,0.188449,0.151954
8,0.191643,0.124529,0.237164,0.113097,0.285823,0.134858


In [18]:
sample_submission = pd.read_csv('./sample_submission.csv')
sample_submission
# for 품목명, predictions in 품목별_predictions.items():
#     sample_submission[품목명] = predictions

# # 결과 저장
# sample_submission.to_csv('./baseline_submission.csv', index=False)


,시점,감자,건고추,깐마늘(국산),대파,무,배추,사과,상추,양파,배
0,TEST_00+1순,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,TEST_00+2순,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,TEST_00+3순,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,TEST_01+1순,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,TEST_01+2순,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...
70,TEST_23+2순,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
71,TEST_23+3순,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
72,TEST_24+1순,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
73,TEST_24+2순,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
